## Курсовой проект "Рекомедательные системы"

### Постановка задачи
Необходимо:

- Сделать бейзлайны
- Сделать модель, подобрать оптимальные параметры
- Для каждого юзера оставить по 5 рекомендаций
- Достичь целевой метрики precision@5 ≈ 0.25 на retail_test1

**! Исключить холодных пользователей**


### Загрузка библиотек

In [310]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares as als
from implicit.bpr import BayesianPersonalizedRanking as bpr

# Модель второго уровня
from catboost import CatBoostClassifier

#import os, sys
#sys.path.insert(1, os.getcwd() + '/src/')

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, postfilter_items, make_unique_recommendations
from src.recommenders import MainRecommender

**Загрузка данных**

In [311]:
DATASET_PATH = '../../retail_train.csv'
ITEM_FEATURES_PATH = '../../product.csv'
USER_FEATURES_PATH = '../../hh_demographic.csv'

In [312]:
data = pd.read_csv(DATASET_PATH)
item_features = pd.read_csv(ITEM_FEATURES_PATH)
user_features = pd.read_csv(USER_FEATURES_PATH)

**Описание датасета:**  
* **user_id** - id покупателя. 
* **backet_id** - номер чека  
* **item_id** - id товара
* **quantity** - количество конкретного товара в одной покупке  
* **sales_value** - стоимость покупи, долл  
* **store_id** - id магазина, где была совершена покупка  
* **retail_disc** - скидка магазина 
* **trans_time** - время покупки (транзакции) 
* **week_no** - номер недели, когда была покупка
* **coupon_disc** -  скидка по купону 
* **coupon_match_disc** - дополнительная скидка

### Подготовка данных

In [313]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

In [314]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [315]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [316]:
user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8


### Опишем полезные функции

In [317]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")
    
def make_recommendations(df_result, recommend_model, N_PREDICT=50, USER_COL='user_id'):
    return df_result[USER_COL].apply(lambda x: recommend_model(x, N=N_PREDICT))

def calc_recall_at_k(df_data, top_k, ACTUAL_COL='actual'):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_precision_at_k(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def rerank(user_id, df, USER_COL='user_id', proba_col_name='proba_item_purchase', N=5):
    return df[df[USER_COL]==user_id].sort_values(proba_col_name, ascending=False).head(N).item_id.tolist()

def get_scores(df_result, recommend_model, N_PREDICT=50, USER_COL='user_id'):
    return df_result[USER_COL].apply(lambda x: recommend_model(x, N=N_PREDICT))

###  Train test split
Делим датасет на 3 части:  
1) обучающий для модели 1 уровня   
2) валидационный для модели 1 уровня = обучающий для модели 2 уровня   
3) валидационный для модели 2 уровня  
   
Модель 1 уровня - MATCHER (сопоставление, нахождение первичных рекомендаций)  
Модель 2 уровня - RANKER (модель для ранжирования, классификационная модель)

In [318]:
VAL_MATCHER_WEEKS = 5
VAL_RANKER_WEEKS = 3

# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]


In [319]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2136728, 12) Users: 2498 Items: 84180
val_matcher
Shape: (141762, 12) Users: 2097 Items: 25770
train_ranker
Shape: (141762, 12) Users: 2097 Items: 25770
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [320]:
#Проведем префильтрацию данных

n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=10000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 84180 to 10001


In [321]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [322]:
# Добавим параметр категории к исходному обучающему датасету для удобства создания новых фичей
df_join_train_matcher = df_join_train_matcher.merge(item_features[['item_id', 'department']], on='item_id', how='inner')
df_join_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,department
0,1285,30407595228,239,852015,1,9.79,334,-2.2,915,35,0.0,0.0,9.79,DRUG GM
1,2076,30768591161,251,852015,1,9.79,388,-2.2,1244,37,-1.0,0.0,9.79,DRUG GM


#### Оставим только пользователей, которые встречаются в тренировочном датасете, чтобы избежать проблемы холодного старта.
* Обработка холодного старта в MainRecommender также добавлена на случай будущих проектов

In [323]:
# ищем общих пользователей
common_users = list(set(data_train_matcher.user_id.values)&(set(data_val_matcher.user_id.values))&set(data_val_ranker.user_id.values))

# оставляем общих пользователей
data_train_matcher = data_train_matcher[data_train_matcher.user_id.isin(common_users)]
data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (546741, 13) Users: 1870 Items: 9997
val_matcher
Shape: (136228, 12) Users: 1870 Items: 25273
train_ranker
Shape: (136228, 12) Users: 1870 Items: 25273
val_ranker
Shape: (114844, 12) Users: 1870 Items: 23952


### Построим baseline

In [324]:
# Инициализируем экземпляр класса MainRecommender

recommender = MainRecommender(data_train_matcher, weighting = True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/9997 [00:00<?, ?it/s]

  0%|          | 0/9997 [00:00<?, ?it/s]

  0%|          | 0/9997 [00:00<?, ?it/s]

  0%|          | 0/9997 [00:00<?, ?it/s]

In [325]:
ACTUAL_COL = 'actual'
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[1005186, 907466, 909497, 940947, 963542, 1067..."
1,6,"[873654, 994928, 1098844, 1122879, 8357613, 98..."


**БЕЙЗЛАЙНЫ**

In [326]:
N_PREDICT = 30

result_eval_matcher['random_recommendations'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                                    recommender.random_recommendation(N = N_PREDICT))

result_eval_matcher['top_popular_recs'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                    recommender.popularity_recommendation(N = N_PREDICT))
result_eval_matcher['weighted_random_recs'] = result_eval_matcher['user_id'].apply(lambda x:\
                                                    recommender.weighted_random_recommendation(N = N_PREDICT))

In [327]:
#Оценка метрик на безйлайнах
top_k_recall = 30
sorted(calc_recall_at_k(result_eval_matcher, top_k_recall), key=lambda x: x[1],reverse=True)

[('top_popular_recs', 0.03002341184325214),
 ('weighted_random_recs', 0.0017247833634099407),
 ('random_recommendations', 0.001113446668857203)]

In [328]:
top_k_precision = 5
sorted(calc_precision_at_k(result_eval_matcher, top_k_precision), key=lambda x: x[1],reverse=True)

[('top_popular_recs', 0.10802139037433155),
 ('weighted_random_recs', 0.003957219251336899),
 ('random_recommendations', 0.0031016042780748665)]

Итак, мы получили значения метрик baseline -ов.   
При построении модели будем сравнивать с ними. **Чем сильнее результат построенной модели будет превосходить лучший из бейзлайнов (top_popular_items), тем лучше эта модель будет**

### Переходим к построению двухуровневой модели
### Построение модели первого уровня Matcher

In [329]:
# Берем в качестве оптимального количества кандидатов 30


N_PREDICT = 30
result_eval_matcher['own_rec'] = make_recommendations(result_eval_matcher, 
                                                       recommender.get_own_recommendations, 
                                                       N_PREDICT=N_PREDICT)

result_eval_matcher['als_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_als_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['bpr_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_bpr_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['bm25_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_bm25_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['tfidf_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_tfidf_recommendations, 
                                                      N_PREDICT=N_PREDICT)

result_eval_matcher['cosine_rec'] = make_recommendations(result_eval_matcher, 
                                                      recommender.get_cosine_recommendations, 
                                                      N_PREDICT=N_PREDICT)



# #similar_users_recommendation, similar_items_recommendation показывают очень низкий результат
# не будем их учитывать в сравнении

In [330]:
result_eval_matcher.head(2)

,user_id,actual,random_recommendations,top_popular_recs,weighted_random_recs,own_rec,als_rec,bpr_rec,bm25_rec,tfidf_rec,cosine_rec
0,1,"[1005186, 907466, 909497, 940947, 963542, 1067...","[1070782, 931688, 896161, 1027474, 877443, 920...","[999999, 1029743, 916122, 1106523, 5569230, 84...","[990688, 829802, 5995603, 8205252, 841469, 852...","[940947, 1004906, 856942, 865456, 5582712, 101...","[965766, 1132771, 940947, 1062002, 1005274, 10...","[916122, 1029743, 844179, 1127831, 1004906, 86...","[940947, 1004906, 1013167, 1127831, 844179, 10...","[900875, 877373, 6391068, 877391, 1109465, 108...","[877373, 900875, 1087895, 9297615, 877391, 856..."
1,6,"[873654, 994928, 1098844, 1122879, 8357613, 98...","[860703, 825006, 6463877, 6463504, 968025, 103...","[999999, 1029743, 916122, 1106523, 5569230, 84...","[9831469, 1007414, 9526630, 835943, 5568817, 5...","[5569230, 965267, 863447, 993638, 1024306, 951...","[1127831, 878996, 1004906, 952163, 13002975, 1...","[916122, 1029743, 844179, 1127831, 1004906, 86...","[878996, 965267, 930118, 1105488, 863447, 1044...","[1004906, 866211, 878996, 1127831, 844179, 916...","[878996, 863447, 13003092, 1098844, 896613, 82..."


In [331]:
top_k_recall = 30
sorted(calc_recall_at_k(result_eval_matcher, top_k_recall), key=lambda x: x[1],reverse=True)

[('own_rec', 0.07783157647707381),
 ('cosine_rec', 0.0771253108814086),
 ('tfidf_rec', 0.07166518919602384),
 ('als_rec', 0.06991505553957553),
 ('bm25_rec', 0.06678114995172708),
 ('top_popular_recs', 0.03002341184325214),
 ('bpr_rec', 0.02913001118108271),
 ('weighted_random_recs', 0.0017247833634099407),
 ('random_recommendations', 0.001113446668857203)]

In [332]:
top_k_precision = 5
sorted(calc_precision_at_k(result_eval_matcher, top_k_precision), key=lambda x: x[1],reverse=True)

[('own_rec', 0.2851336898395722),
 ('cosine_rec', 0.24727272727272728),
 ('bm25_rec', 0.1918716577540107),
 ('tfidf_rec', 0.17871657754010697),
 ('als_rec', 0.13593582887700537),
 ('top_popular_recs', 0.10802139037433155),
 ('bpr_rec', 0.09989304812834225),
 ('weighted_random_recs', 0.003957219251336899),
 ('random_recommendations', 0.0031016042780748665)]

Очевидно, лучшие результаты показывает модель на основе **предыдущих покупок пользователя** 

### Подготовка датасета и генерация признаков для модели 2 уровня (модели ранжирования)

In [333]:
# берем пользователей из трейна для модели ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]


# собираем для каждого юзера кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [334]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [335]:
# соберем итоговый датафрейм с комбинациями user_id - item_id
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [336]:
df_match_candidates.head(3)

,user_id,item_id
0,1827,907631
0,1827,940947
0,1827,1029743


In [337]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (56100, 2) Users: 1870 Items: 4324


In [338]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # здесь только фактические покупки 

df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

# все, что не фактические покупки - заполняем нулями
df_ranker_train['target'].fillna(0, inplace= True)

df_ranker_train.head()

,user_id,item_id,target
0,1827,907631,0.0
1,1827,940947,0.0
2,1827,1029743,1.0
4,1827,5568378,0.0
5,1827,854405,0.0


In [339]:
# Соотношение классов:

df_ranker_train.target.value_counts()

0.0    48109
1.0     7839
Name: target, dtype: int64

Очевидно, что объектов 0 класса гораздо больше, чем 1
Вероятно это связано с количеством кандидатов

In [340]:
# Присоединим к новому тренировочному датасету фичи юзеров и товаров - создадим общий тренировочный датасет

df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,1827,907631,0.0,1039,GROCERY,National,FROZEN PIZZA,SNACKS/APPETIZERS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1827,940947,0.0,2082,MEAT-PCKGD,National,HEAT/SERVE,ENTREES,24 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Генерация новых признаков

In [341]:
# Добавим параметр категории к исходному обучающему датасету для удобства создания новых фичей
data_department = data_train_ranker.merge(item_features[['item_id', 'department']], on='item_id', how='inner')
data_department.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,department
0,1827,40702967646,601,891141,2,2.73,33923,0.0,7,87,0.0,0.0,PRODUCE
1,496,40739402373,603,891141,1,1.83,445,0.0,2226,87,0.0,0.0,PRODUCE


In [342]:
# Добавим признаки

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().\
                                        rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
                                        rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().\
                                        rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().\
                                        rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().\
                                        rename('total_user_sales_value'), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
                rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().\
                rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().\
            rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().\
            rename('user_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().\
                rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().\
                rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)




**Добавление дополнительных признаков**

In [343]:
''' ДЛЯ ЮЗЕРОВ '''

# Средний чек
users_sales = data_train_ranker.groupby(USER_COL)['sales_value'].mean().reset_index()
users_sales.rename(columns={'sales_value': 'avg_cheque'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_sales[['user_id', 'avg_cheque']], on='user_id', how='left')

# Количество уникальных категорий покупателя
users_departments = data_department.groupby(USER_COL)['department'].nunique().reset_index()
users_departments.rename(columns = {'department':'users_unique_departments'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_departments, on='user_id', how='left')

# Среднее время покупки
bought_time = data_train_ranker.groupby(USER_COL)['trans_time'].mean().reset_index()
bought_time.rename(columns = {'trans_time':'mean_trans_time_by_user'}, inplace=True)
df_ranker_train = df_ranker_train.merge(bought_time, on='user_id', how='left')

# Средний чек корзины 
baskets_sales_value = data_train_ranker.groupby([USER_COL,'basket_id'])['sales_value'].mean().reset_index()
mean_basket_sales_value = baskets_sales_value.groupby(USER_COL)['sales_value'].mean().reset_index()
mean_basket_sales_value.rename(columns = {'sales_value':'mean_sales_value_per_basket'}, inplace=True)
df_ranker_train = df_ranker_train.merge(mean_basket_sales_value, on='user_id', how='left')

# Количество купленных уникальных товаров 
unique_bought_items = data_train_ranker.groupby(USER_COL)[ITEM_COL].nunique().reset_index()
unique_bought_items.rename(columns = {'item_id':'unique_bought_items'}, inplace=True)
df_ranker_train = df_ranker_train.merge(unique_bought_items, on='user_id', how='left')



# Среднее количество уникальных категорий в корзине
users_baskets = data_department.groupby([USER_COL, 'basket_id'])['department'].nunique().reset_index()
users_baskets = users_baskets.groupby(USER_COL)['department'].mean().reset_index()
users_baskets.rename(columns={'department': 'avg_basket_department'}, inplace=True)
df_ranker_train = df_ranker_train.merge(users_baskets[['user_id', 'avg_basket_department']], on='user_id', how='left')

# Средняя сумма покупки в категории
department_sales = data_department.groupby('department')['sales_value'].mean().reset_index()
department_sales.rename(columns={'sales_value': 'mean_sales_value_category'}, inplace=True)
df_ranker_train = df_ranker_train.merge(department_sales, on='department', how='left')

# Средная цена купленных товаров пользователем
users_sales = data_train_ranker.groupby(USER_COL)[['sales_value', 'quantity']].sum().reset_index()
users_sales['avg_price'] = users_sales['sales_value'] / users_sales['quantity']
df_ranker_train = df_ranker_train.merge(users_sales[['user_id', 'avg_price']], on='user_id', how='left')


In [344]:
''' ДЛЯ ТОВАРОВ '''

# Среднее количество покупок товара в неделю
num_purchase_week = data_train_ranker.groupby(ITEM_COL).agg({'week_no': 'nunique', 'quantity': 'sum'}).reset_index()
num_purchase_week['avg_num_purchases_week'] = num_purchase_week['quantity'] / num_purchase_week['week_no']
df_ranker_train = df_ranker_train.merge(num_purchase_week[['item_id', 'avg_num_purchases_week']], on='item_id', how='left')
df_ranker_train['avg_num_purchases_week'].fillna(0, inplace= True)



# Цена товара
items_sales = data_department.groupby(ITEM_COL)[['sales_value', 'quantity']].sum().reset_index()
items_sales['price'] = items_sales['sales_value'] / items_sales['quantity']
items_sales['price'].fillna(0, inplace=True)
df_ranker_train = df_ranker_train.merge(items_sales[['item_id', 'price']], on='item_id', how='left')


# Среднее время покупки товара
bought_item_time = data_train_ranker.groupby(ITEM_COL)['trans_time'].mean().reset_index()
bought_item_time.rename(columns = {'trans_time':'mean_trans_time_by_item'}, inplace=True)
da_ranker_train = df_ranker_train.merge(bought_item_time, on = 'item_id', how = 'left')


# Количество магазинов, где есть товар
items_stores = data_department.groupby(ITEM_COL)['store_id'].sum().reset_index()
items_stores.rename(columns={'store_id': 'n_stores_with_item'}, inplace=True)
items_stores['n_stores_with_item'].fillna(0, inplace = True)
df_ranker_train = df_ranker_train.merge(items_stores, on=ITEM_COL, how='left')

# Количество уникальных магазинов, где есть товар
items_stores = data_department.groupby(ITEM_COL)['store_id'].nunique().reset_index()
items_stores.rename(columns={'store_id': 'n_unique_stores_with_item'}, inplace=True)
items_stores['n_unique_stores_with_item'].fillna(0, inplace = True)
df_ranker_train = df_ranker_train.merge(items_stores, on=ITEM_COL, how='left')


In [345]:
# Построим признак, отражающий средний интервал между покупками пользователя.
users_days = df_join_train_matcher.groupby(USER_COL)['day'].unique().reset_index()
users_days['day'] = users_days['day'].apply(lambda x: sorted(x))
users_days.head()

,user_id,day
0,1,"[246, 263, 274, 276, 282, 291, 300, 311, 317, ..."
1,2,"[239, 249, 263, 291, 318, 335, 345, 415, 432, ..."
2,3,"[242, 243, 248, 253, 260, 266, 291, 304, 320, ..."
3,4,"[244, 248, 251, 264, 288, 292, 302, 321, 328, ..."
4,5,"[258, 300, 320, 334, 418, 433, 449, 450, 460, ..."


In [346]:
def avg_ndays(days):
    diff = 0
    if len(days) > 1:
        for i in range(len(days) - 1):
            diff += days[i+1] - days[i]
        return diff / (len(days) - 1)
    else:
        return 0
    
users_days['avg_interval'] = users_days['day'].apply(avg_ndays)

df_ranker_train = df_ranker_train.merge(users_days[['user_id', 'avg_interval']], on='user_id', how='left')
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,mean_sales_value_per_basket,unique_bought_items,avg_basket_department,mean_sales_value_category,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval
0,1827,907631,0.0,1039,GROCERY,National,FROZEN PIZZA,SNACKS/APPETIZERS,,NaN,...,2.965362,28,2.4,2.545290,2.208947,16.0,2.985000,276749.0,39.0,10.333333
1,1827,940947,0.0,2082,MEAT-PCKGD,National,HEAT/SERVE,ENTREES,24 OZ,NaN,...,2.965362,28,2.4,3.825916,2.208947,19.4,2.650825,383770.0,48.0,10.333333


Построим признак, в котором будет закодировано место товара в пяти последних покупках клиента.

In [347]:
users_items = data_train_ranker.groupby(USER_COL)[ITEM_COL].apply(list).reset_index()
users_items['item_id'] = users_items['item_id'].apply(lambda x: x[-5:])
users_items.head()

,user_id,item_id
0,1,"[5577022, 8293439, 9526676, 9527558, 10149640]"
1,6,"[1099058, 895268, 1017061, 1082185, 1119051]"
2,7,"[9837501, 12524016, 13072715, 13987153, 13987338]"
3,8,"[924610, 999142, 1080014, 1121694, 1130286]"
4,9,"[7467081, 10150194, 10457112, 12132773, 12171886]"


In [348]:
users_items.loc[users_items['user_id'] == 67, 'item_id'].item()

[1135408, 9194664, 9526666, 10182813, 12385050]

In [349]:
def code_last_sales(x, df=users_items):
    last_sales = df.loc[df['user_id'] == x[0], 'item_id'].item()
    code = str()
    last_sales.reverse()
    for item in last_sales:
        code += '1' if item == x[1] else '0'
    return code

df_ranker_train['Last5sales'] = df_ranker_train[[USER_COL, ITEM_COL]].apply(code_last_sales, axis=1)
df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,unique_bought_items,avg_basket_department,mean_sales_value_category,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval,Last5sales
0,1827,907631,0.0,1039,GROCERY,National,FROZEN PIZZA,SNACKS/APPETIZERS,,NaN,...,28,2.4,2.545290,2.208947,16.0,2.985000,276749.0,39.0,10.333333,00000
1,1827,940947,0.0,2082,MEAT-PCKGD,National,HEAT/SERVE,ENTREES,24 OZ,NaN,...,28,2.4,3.825916,2.208947,19.4,2.650825,383770.0,48.0,10.333333,00000


In [350]:
# Проверим наличие пропусков
df_ranker_train.isnull().sum()

user_id                            0
item_id                            0
target                             0
manufacturer                       0
department                         0
brand                              0
commodity_desc                     0
sub_commodity_desc                 0
curr_size_of_product               0
age_desc                       33239
marital_status_code            33239
income_desc                    33239
homeowner_desc                 33239
hh_comp_desc                   33239
household_size_desc            33239
kid_category_desc              33239
total_item_sales_value             0
total_quantity_value               0
item_freq                          0
user_freq                          0
total_user_sales_value             0
item_quantity_per_week             0
user_quantity_per_week             0
item_quantity_per_basket           0
user_quantity_per_basket           0
item_freq_per_basket               0
user_freq_per_basket               0
a

In [351]:
#Заполним пропуски у количественных признаков
df_ranker_train[['price', 'n_stores_with_item', 'n_unique_stores_with_item']].fillna(0, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


**Разделение на X_train и y_train и обучение модели**

In [352]:
X_train = df_ranker_train.drop(columns = ['target',
                                          'total_quantity_value',
                                          'user_quantity_per_week',
                                          'mean_sales_value_category',
                                          'item_quantity_per_basket',
                                          
                                         ])

#Убрала количественные признаки в весом feature_importances ниже 1 (категориальные оставим)
y_train = df_ranker_train[['target']]

In [353]:
X_train.head(2)

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,...,mean_sales_value_per_basket,unique_bought_items,avg_basket_department,avg_price,avg_num_purchases_week,price,n_stores_with_item,n_unique_stores_with_item,avg_interval,Last5sales
0,1827,907631,1039,GROCERY,National,FROZEN PIZZA,SNACKS/APPETIZERS,,NaN,NaN,...,2.965362,28,2.4,2.208947,16.0,2.985000,276749.0,39.0,10.333333,00000
1,1827,940947,2082,MEAT-PCKGD,National,HEAT/SERVE,ENTREES,24 OZ,NaN,NaN,...,2.965362,28,2.4,2.208947,19.4,2.650825,383770.0,48.0,10.333333,00000


In [354]:
# Обработка категориальных признаков

cat_feats = [
          'department',
         'brand',
         'commodity_desc',
         'sub_commodity_desc',
         'curr_size_of_product',
         'age_desc',
         'marital_status_code',
         'income_desc',
         'homeowner_desc',
         'hh_comp_desc',
         'household_size_desc',
         'kid_category_desc',
         'Last5sales'
    
]


for col in cat_feats:
    X_train[col].fillna(0, inplace=True)

X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'Last5sales']

In [355]:
#Рассчитаем дисбаланс классов - насколько объектов 0 класса больше, чем объектов 1 класса
disbalance = y_train.value_counts()[0] / y_train.value_counts()[1]
disbalance

6.137134838627376

**Пострение модели**

In [356]:
#ПАРАМЕТРЫ БЫЛИ ПОДОБРАНЫ ПОИСКОМ ПО СЕТКЕ, НЕ ВКЛЮЧЕНО В ИТОГОВЫЙ ПРОЕКТ ДЛЯ СОКРАЩЕНИЯ ВРЕМЕНИ РАБОТЫ НОУТБУКА

ctb = CatBoostClassifier(learning_rate=0.1,
                        max_depth=12,
                        n_estimators=550,
                        random_state=42, 
                        cat_features=cat_feats, 
                        class_weights=[1, disbalance],
                        silent=True)

ctb.fit(X_train, y_train)

train_preds = ctb.predict_proba(X_train)

In [357]:
# Изучим важность признаков в модели
fi = pd.DataFrame(ctb.feature_importances_, index=X_train.columns, columns=['importance'])
fi.sort_values(by='importance', ascending=False)

,importance
unique_bought_items,6.105479
item_id,5.730828
user_id,5.312913
mean_trans_time_by_user,5.021458
sub_commodity_desc,4.947824
avg_basket_department,4.905978
commodity_desc,4.368842
price,4.133700
department,3.832607
avg_num_purchases_week,3.651498


In [358]:
# Оценим качество построенной модели 
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [359]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]

#Добавляем сначала предсказания (рекомендации) модели 1 уровня
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

Сделаем переранжирование рекомендаций на основе результатов классификации модели 2 уровня

In [360]:
def rerank(user_id, N):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].\
            sort_values('proba_item_purchase', ascending=False).head(N).item_id.tolist()

In [361]:
TOPK_PRECISION = 5

result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                                                            rerank(user_id, N=5))

# Оставляем заведомо больше ранжированных предсказаний для постфильтрации
result_eval_ranker['postfiltered_reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                                     postfilter_items(rerank(user_id, N=20), item_features = item_features, N=5))
result_eval_ranker['uniquue_reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id:\
                         make_unique_recommendations(rerank(user_id, N=20), N=5))

print(*sorted(calc_precision_at_k(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.28363636363636363)
('uniquue_reranked_own_rec', 0.28363636363636363)
('own_rec', 0.2621390374331551)
('postfiltered_reranked_own_rec', 0.23390374331550803)


Удалив дубликаты в df_train_ranker на этапе подготовки датасета, мы сразу получили уникальные товары, поэтому применяя фильтрацию товаров, чтобы остались только уникальные - метрика не меняется, как и товары.  
  
Когда мы делаем постфильтрацию товаров таким образом, чтобы каждый товар был из отдельной категории, метрика существенно падает -> постфильтрация ухудщает результат

**Проверим, везде ли одинаковое количество рекомендаций**  
если их кажется меньше, то необходимо будет реализовать дополнение

In [362]:
for num, row in enumerate(result_eval_ranker['own_rec']):
    if len(row) != 30:
        print(num)

In [363]:
for num, row in enumerate(result_eval_ranker['reranked_own_rec']):
    if len(row) != 5:
        print(num)

### Оценка на тестовом наборе данных

In [364]:
df_test = pd.read_csv('../retail_test1.csv')
df_test.shape

(88734, 12)

In [365]:
df_test = df_test[df_test.user_id.isin(common_users)]

In [366]:
final_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
final_test.columns=[USER_COL, ACTUAL_COL]

In [367]:
final_test['own_rec'] = final_test[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
final_test['reranked_own_rec'] = final_test[USER_COL].apply(lambda user_id: rerank(user_id, N=5))

print(*sorted(calc_precision_at_k(final_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.24085626911314986)
('own_rec', 0.22422018348623854)


In [372]:
# Проверим, есть ли строки где количество предсказаний != 5
for num, row in enumerate(final_test['reranked_own_rec']):
    if len(row) != 5:
        print(num)

In [369]:
# Сохраним рекомендации
recommendations = final_test[[USER_COL, 'reranked_own_rec']]
recommendations.rename(columns = {'reranked_own_rec' : 'recommendations'}, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [370]:
recommendations.to_csv('recommendations.csv', index=False)
recommendations.head(2)

,user_id,recommendations
0,1,"[5577022, 8293439, 10149640, 9527558, 9297615]"
1,6,"[6548453, 1098844, 1024306, 878996, 1029743]"
